<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW1/blob/main/NLP_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Natural Language Processing-Homework1

####**Student name** : Amirahmad Shafiee<br/>**Student number** : 99104027<br/>**Chosen task** : NER
In this notebook we aim to do common but rather essential preprocess actions on some text chosen from the persian literature world. After doing so, a usual NLP task would be operated on the processed text.

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/NLP'
/content


##Section A: Extracting data

